## 1. Импорт библиотек

In [1]:
import os

import pandas as pd

from utils.cleaning import normalize_street, normalize_district, drop_outlers
from utils.features import get_total_price

## 2. Чтение и подготовка данных

Задаём пути к файлам, параметры чтения CSV и загружаем данные с проверкой ошибок.

In [2]:
HOME = os.getcwd().removesuffix("/notebooks")

In [3]:
# Настраиваем словари с параметрами чтения CSV: путь, кодировка, дата-колонки
rides_options = {
    "filepath_or_buffer": f"{HOME}/data/rides.csv",
    "encoding": "utf-8",
    "parse_dates": ["Start Date", "End Date"],
}

weather_options = {
    "filepath_or_buffer": f"{HOME}/data/weather.csv",
    "encoding": "utf-8",
    "parse_dates": ["Datetime"]
}

try:
    rides_data = pd.read_csv(**rides_options)
    weather_data = pd.read_csv(**weather_options)
except pd.errors.ParserError as e:
    print(f"Error: {e}")
    rides_data = pd.read_csv(**rides_options, sep=";")
    weather_data = pd.read_csv(**weather_options, sep=";")

## 3. Переименование колонок

Приводим названия колонок к snake_case для удобства работы.

In [4]:
rides_data.rename(lambda col: col.strip().lower().replace(" ", "_"), axis="columns", inplace=True)

weather_data.rename(lambda col: col.strip().lower().replace(" ", "_"), axis="columns", inplace=True)

## 4. Очистка данных о поездках

### 4.1. Преобразование типов

Колонка `promo` бинарная, поэтому преобразуется в категориальный тип (`category`).

In [5]:
rides_data["promo"] = rides_data["promo"].astype(object)

### 4.2. Удаление дубликатов

In [6]:
rides_data = rides_data.drop_duplicates()

rides_data.duplicated().sum()

np.int64(0)

### 4.3. Нормализация адресов и районов

Применяем функции из модуля `cleaning` для стандартизации названий улиц и районов.

In [7]:
rides_data["start_location"] = rides_data["start_location"].apply(normalize_street)
rides_data["end_location"] = rides_data["end_location"].apply(normalize_street)

In [8]:
rides_data["start_district"] = rides_data["start_district"].apply(normalize_district)
rides_data["end_district"] = rides_data["end_district"].apply(normalize_district)

### 4.4. Удаление пропусков
На первичном анализе было выявлено, что доля пропусков в данных менее 1%, поэтому их строки можно безопасно удалить.


In [9]:
rides_data = rides_data.dropna()

rides_data.isnull().sum()

id                0
start_date        0
end_date          0
start_location    0
start_district    0
end_location      0
end_district      0
distance          0
promo             0
dtype: int64

### 4.5. Удаление экстремальных значением

Применяем метод межквартильного размаха (IQR) для очистки датасета от экстремальных значений в колонке `distance`.

In [10]:
rides_data = drop_outlers(rides_data, "distance", k=3)
rides_data = rides_data[rides_data["distance"] >= 200]

### 4.6. Создание новых признаков

Добавляем итоговое время поездки.

In [11]:
td = rides_data["end_date"] - rides_data["start_date"]

rides_data["duration_minutes"] = (td.dt.total_seconds() / 60).round().astype(int)

Добавляем день недели и перемещаем колонку на удобную позицию.

In [12]:
rides_data["day_of_week"] = rides_data["start_date"].dt.dayofweek

col = rides_data.pop('day_of_week')
rides_data.insert(3, 'day_of_week', col)

Добавляем итоговую стоимость поездки.

In [13]:
rides_data["total_price"] = rides_data.apply(get_total_price, axis=1)